# Assignment 7

### Part A

In [2]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [13]:
def expected_return(P, rf, mkt_rp):
    exp_return = 0
    total = P['Amount Invested'].sum()
    length = len(P)
    for i in range(length):
        exp_return += (rf + P.at[i, 'Beta'] * mkt_rp) * (P.at[i, 'Amount Invested'] / total)

    return exp_return


0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [14]:
def volatility(P, cor):
    total = P['Amount Invested'].sum()
    volatility = 0
    length = len(P)
    for i in range(length):
        w = pow(P.at[i, 'Amount Invested'] / total, 2)
        vol = pow(P.at[i, 'Volatility'], 2)
        volatility += w * vol
        v = cor * length
    for i in range(length):
        v *= (P.at[i, 'Volatility']) * (P.at[i, 'Amount Invested'] / total)
    volatility += v
    return pow(volatility, 1/2)


0.24083189157584595

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [15]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    risky = vol / mkt_vol
    return rf + risky * mkt_rp

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [3]:
def CAPM_beta(return_path, factors_path):
    df_return = pd.read_csv(return_path)
    df_factors = pd.read_csv(factors_path)
    df_return['date'] = (df_return['year'].astype(str) + df_return['month'].astype(str).str.zfill(2)).astype(int)
    df_merge = pd.merge(df_return, df_factors, on='date')
    df_merge['Mkt-RF'] /= 100
    df_merge['RF'] /= 100

    df_merge['excess_return_firm'] = df_merge['RET'] - df_merge['RF']
    df_merge['excess_return_market'] = df_merge['Mkt-RF']
    
    exp_returns = df_merge.groupby('CUSIP')['RET'].mean().reset_index(name='EXP_RETURN')
    merged = df_merge.groupby('CUSIP')
    beta = merged.apply(lambda x: np.cov(x['excess_return_firm'], x['excess_return_market'])[0, 1] / x['excess_return_market'].var()).reset_index(name='BETA')
    
    df = pd.merge(exp_returns, beta, on='CUSIP')

    output = pd.DataFrame(data = {
        "CUSIP" : df['CUSIP'],
        "EXP_RETURN" : df['EXP_RETURN'],
        "BETA" : df['BETA'],
    })
    
    return output

return_path = '../../datafiles/A7/monthlycrsp_mod7.csv'
factors_path = '../../datafiles/A7/factors.CSV'

# Takes a while (~10 min)
data = CAPM_beta(return_path, factors_path)
data

CUSIP  EXP_RETURN       BETA
0  36020   -0.032023  -0.483703
1  36021    0.028259  -1.124363
2  36022    0.048830  -5.149374
3  36023    0.089881  10.187062

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [17]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    c = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    return c[1]


0.007981588120535638